In [1]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
import plotly
import plotly.graph_objects as go
from matplotlib import pyplot as plt
import plotly.express as px
from pathlib import Path
import warnings
import config
import preprocess
from custom_metrics import LoglossObjective_loop, LoglossObjective_np
from custom_metrics import FocalLossFormulas, AsymmetricLossFormulas
from custom_metrics import get_simplified_derivative
from collections import defaultdict
from catboost import CatBoostClassifier, Pool

sys.path.append('./cython_loss')

os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'
# import featuretools as ft

warnings.filterwarnings("ignore", category=DeprecationWarning)

pd.set_option('display.max_rows', 120)
plt.style.use("dark_background")
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams["hist.bins"] = 50
print('cur_dir', Path.cwd())

cur_dir /home/sergey/mnt/st1500/Usr/Sergey/TheJob/Otus/ML_advanced2020/final_project


/home/sergey/anaconda3/envs/final_project/lib/python3.8/site-packages/featuretools_update_checker/functions.py:34: UserWarning: Featuretools is out-of-date: installed == 0.23.1, latest == 0.23.2
  warnings.warn(msg)


In [2]:
data = pd.read_csv("/home/sergey/mnt/4.5Tb/Downloads/otus_final_input/data_encoded.csv",
                   low_memory=False,
                   ).astype(config.dtypes)
data = preprocess.preprocess(data)
print("", data.shape)
data.head()

 (274702, 13)


,DealDurationDays,DealDurationMonths,Client_Id,DealDate,ValueDate,MaturityDate,Deal_characteristics_1,Deal_characteristics_2,Deal_characteristics_3,Deal_characteristics_4,Client_characteristics_1,Client_characteristics_2,target
0,8.0,0,16140,2015-01-12,2015-01-12,2015-01-20,7.018746,0.0000,0.140368,0,15,1759,1
1,53.0,1,21990,2015-01-12,2015-01-12,2015-03-06,0.306237,0.1146,0.087720,2,15,1759,1
2,92.0,3,18607,2015-01-12,2015-01-12,2015-04-14,0.306237,0.1201,0.054139,2,15,1759,1
3,1.0,0,20500,2015-01-12,2015-01-12,2015-01-13,2.985808,0.0896,0.100056,2,15,1759,1
4,1.0,0,4041,2015-01-12,2015-01-12,2015-01-13,0.418013,0.0805,0.168161,2,15,1759,1


In [3]:
# print(data.columns)

In [4]:
X_val = data.groupby(data.Client_Id).tail(1)
y_val = X_val.pop('target')
X_train = data.drop(labels=X_val.index)
y_train = X_train.pop('target')

## активность клиентов, простые статистики

In [5]:
client_id_activity = X_train.groupby(data.Client_Id).agg(['count', 'mean', 'median'])
client_id_activity.sort_values(by=('DealDurationDays', 'count'), ascending=False)

активность клиентов, простые статистики


DealDurationDays                   DealDurationMonths            \
                     count       mean median              count      mean   
Client_Id                                                                   
0                    16510  13.290612    7.0              16510  0.245548   
7596                  3353   1.526991    1.0               3353  0.000000   
17708                 2158   1.994903    1.0               2158  0.002780   
8269                  1797   5.678353    6.0               1797  0.000000   
11790                 1439  29.776928   26.0               1439  0.503127   
...                    ...        ...    ...                ...       ...   
1476                     1  28.000000   28.0                  1  0.000000   
10884                    1   7.000000    7.0                  1  0.000000   
1462                     1   3.000000    3.0                  1  0.000000   
5381                     1  13.000000   13.0                  1  0.000000   
6840                     1  33.000000   33.0                  1  1.000000   

                 Client_Id               Deal_characteristics_1  ...  \
          median     count   mean median                  count  ...   
Client_Id                                                        ...   
0            0.0     16510      0      0                  16510  ...   
7596         0.0      3353   7596   7596                   3353  ...   
17708        0.0      2158  17708  17708                   2158  ...   
8269         0.0      1797   8269   8269                   1797  ...   
11790        0.0      1439  11790  11790                   1439  ...   
...          ...       ...    ...    ...                    ...  ...   
1476         0.0         1   1476   1476                      1  ...   
10884        0.0         1  10884  10884                      1  ...   
1462         0.0         1   1462   1462                      1  ...   
5381         0.0         1   5381   5381                      1  ...   
6840         1.0         1   6840   6840                      1  ...   

          Deal_characteristics_3 Deal_characteristics_4                   \
                          median                  count      mean median   
Client_Id                                                                  
0                       0.074074                  16510  1.677832      2   
7596                    0.067925                   3353  2.000000      2   
17708                   0.076577                   2158  1.000000      1   
8269                    0.080000                   1797  0.000000      0   
11790                   0.079261                   1439  0.000000      0   
...                          ...                    ...       ...    ...   
1476                    0.127649                      1  0.000000      0   
10884                   0.109683                      1  2.000000      2   
1462                    0.120395                      1  2.000000      2   
5381                    0.078762                      1  2.000000      2   
6840                    0.086843                      1  2.000000      2   

          Client_characteristics_1                   Client_characteristics_2  \
                             count       mean median                    count   
Client_Id                                                                       
0                            16510  10.497638   13.0                    16510   
7596                          3353   7.232627    7.0                     3353   
17708                         2158  15.000000   15.0                     2158   
8269                          1797  10.648303   15.0                     1797   
11790                         1439  10.970813   15.0                     1439   
...                            ...        ...    ...                      ...   
1476                             1  15.000000   15.0                        1   
10884                            1  15.000000  

In [6]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)

    X_train = X_train.merge(client_id_activity, how='left', left_on='Client_Id', right_index=True).dropna()
    X_val = X_val.merge(client_id_activity, how='left', left_on='Client_Id', right_index=True)  # .dropna()
# y_val = y_val[X_val.index]

In [7]:
print('data.shape', data.shape)
print('X_train.shape', X_train.shape)
print('X_val.shape', X_val.shape)
print('data.shape[0] - X_val.shape[0] = ', data.shape[0] - X_val.shape[0])
print('data.shape[0] -  X_train.shape[0] - X_val.shape[0] = ', data.shape[0] - X_train.shape[0] - X_val.shape[0])
print(X_train.Client_Id.nunique(), X_val.Client_Id.nunique(), X_train.Client_Id.nunique() - X_val.Client_Id.nunique())

data.shape (274702, 13)
X_train.shape (257180, 39)
X_val.shape (17522, 39)
data.shape[0] - X_val.shape[0] =  257180
data.shape[0] -  X_train.shape[0] - X_val.shape[0] =  0
13050 17522 -4472


In [8]:
train_pool = Pool(data=X_train, label=y_train,
                  cat_features=config.categorical_dtypes_ft.keys())
val_pool = Pool(data=X_val, label=y_val,
                cat_features=config.categorical_dtypes_ft.keys())

## родная Logloss, catboost

In [9]:
%%time

iterations = 200
params = config.params
params.update({
    'loss_function': 'Logloss',
    'iterations': iterations,
    'verbose': iterations // 5,
})
clf_logloss_ref = CatBoostClassifier(**params)
clf_logloss_ref.fit(X=train_pool,
        eval_set=val_pool,
        use_best_model=True)


0:	learn: 0.7196265	test: 0.6101332	best: 0.6101332 (0)	total: 247ms	remaining: 4.69s
4:	learn: 0.7246614	test: 0.6145145	best: 0.6145145 (3)	total: 954ms	remaining: 2.86s
8:	learn: 0.7252500	test: 0.6145384	best: 0.6145384 (8)	total: 1.69s	remaining: 2.07s
12:	learn: 0.7252766	test: 0.6145491	best: 0.6145491 (12)	total: 2.45s	remaining: 1.32s
16:	learn: 0.7254810	test: 0.6147518	best: 0.6147518 (15)	total: 3.28s	remaining: 579ms
19:	learn: 0.7257810	test: 0.6147749	best: 0.6147749 (19)	total: 3.88s	remaining: 0us

bestTest = 0.6147749453
bestIteration = 19

CPU times: user 12.8 s, sys: 829 ms, total: 13.6 s
Wall time: 4.51 s


##### внешняя реализация Logloss, numpy, catboost

In [10]:
%%time
params = config.params
params.update({
    'loss_function': LoglossObjective_np(),
    'iterations': iterations,
    'verbose': iterations // 5,
})
clf_logloss_custom_np = CatBoostClassifier(**params)

clf_logloss_custom_np.fit(X=train_pool,
        eval_set=val_pool,
        use_best_model=True)
print(f'Функция потерь logloss реализована через numpy, \n'
      f'работает экспоненцильно медленнее, чем родная от catboost\n'
      f'на 20 итерациях раз в 5 хуже, на 200 - в 400 раз медленнее\n'
      f'но намного быстрее, чем если делать с циклом внутри \n'
      f'также сходится медленнее, чем родная. Сходимость одинаковая с реализацией через цикл \n'
      f'о сходимости есть топик на stackoverflow здесь \n'
      f'https://stackoverflow.com/questions/63104119/catboost-custom-loss-function \n'
      f'')

## внешняя реализация Logloss, простой Python цикл,  catboost

In [11]:
%%time
params = config.params
params.update({
    'loss_function': LoglossObjective_loop(),
    'iterations': iterations,
    'verbose': iterations // 5,
})
clf_logloss_custom_loop = CatBoostClassifier(**params)

clf_logloss_custom_loop.fit(X=train_pool,
        eval_set=val_pool,
        use_best_model=True)
print(f'Здесь функция потерь logloss реализована через цикл, очень медленно выполняется, \n'
      f'примерно в 8 раз медленнее, чем если реализовать через numpy \n'
      f'и в 30 раз медленнее, чем референсная, родная catboost \n'
      f'также сходится медленнее, чем родная. Сходимость одинаковая с реализацией через numpy ')

print('clf_logloss_custom_loop.get_evals_result()')
loop_res = clf_logloss_custom_loop.get_evals_result()
loop_res

## внешняя реализация FocalLoss,
Python, Numpy, первая и вторая производные получены в sympy,  catboost

Формулы функции потерь обозначены L 
первая и вторая производные der1 и der2

AsymmetricLoss несколко сложнее, чем FocalLoss, 


In [14]:
FocalLossFormulas()

Формулы функции потерь обозначены L 
первая и вторая производные der1 и der2


Eq(p, exp(x)/(exp(x) + 1))

Eq(L_plus, (1 - exp(x)/(exp(x) + 1))**gamma*log(exp(x)/(exp(x) + 1)))

Eq(L_minus, (exp(x)/(exp(x) + 1))**gamma*log(1 - exp(x)/(exp(x) + 1)))

Eq(L, -y*(1 - exp(x)/(exp(x) + 1))**gamma*log(exp(x)/(exp(x) + 1)) - (exp(x)/(exp(x) + 1))**gamma*(1 - y)*log(1 - exp(x)/(exp(x) + 1)))

Eq(L__der1, -gamma*y*(1 - exp(x)/(exp(x) + 1))**gamma*(-exp(x)/(exp(x) + 1) + exp(2*x)/(exp(x) + 1)**2)*log(exp(x)/(exp(x) + 1))/(1 - exp(x)/(exp(x) + 1)) - gamma*(exp(x)/(exp(x) + 1))**gamma*(1 - y)*(exp(x)/(exp(x) + 1) - exp(2*x)/(exp(x) + 1)**2)*(exp(x) + 1)*exp(-x)*log(1 - exp(x)/(exp(x) + 1)) - y*(1 - exp(x)/(exp(x) + 1))**gamma*(exp(x)/(exp(x) + 1) - exp(2*x)/(exp(x) + 1)**2)*(exp(x) + 1)*exp(-x) - (exp(x)/(exp(x) + 1))**gamma*(1 - y)*(-exp(x)/(exp(x) + 1) + exp(2*x)/(exp(x) + 1)**2)/(1 - exp(x)/(exp(x) + 1)))

Eq(L__der2, -gamma**2*y*(1 - exp(x)/(exp(x) + 1))**gamma*(-exp(x)/(exp(x) + 1) + exp(2*x)/(exp(x) + 1)**2)**2*log(exp(x)/(exp(x) + 1))/(1 - exp(x)/(exp(x) + 1))**2 - gamma**2*(exp(x)/(exp(x) + 1))**gamma*(1 - y)*(exp(x)/(exp(x) + 1) - exp(2*x)/(exp(x) + 1)**2)**2*(exp(x) + 1)**2*exp(-2*x)*log(1 - exp(x)/(exp(x) + 1)) - 2*gamma*y*(1 - exp(x)/(exp(x) + 1))**gamma*(-exp(x)/(exp(x) + 1) + exp(2*x)/(exp(x) + 1)**2)*(exp(x)/(exp(x) + 1) - exp(2*x)/(exp(x) + 1)**2)*(exp(x) + 1)*exp(-x)/(1 - exp(x)/(exp(x) + 1)) - gamma*y*(1 - exp(x)/(exp(x) + 1))**gamma*(-exp(x)/(exp(x) + 1) + 3*exp(2*x)/(exp(x) + 1)**2 - 2*exp(3*x)/(exp(x) + 1)**3)*log(exp(x)/(exp(x) + 1))/(1 - exp(x)/(exp(x) + 1)) - gamma*y*(1 - exp(x)/(exp(x) + 1))**gamma*(-exp(x)/(exp(x) + 1) + exp(2*x)/(exp(x) + 1)**2)*(exp(x)/(exp(x) + 1) - exp(2*x)/(exp(x) + 1)**2)*log(exp(x)/(exp(x) + 1))/(1 - exp(x)/(exp(x) + 1))**2 + gamma*(exp(x)/(exp(x) + 1))**gamma*(1 - y)*(exp(x)/(exp(x) + 1) - exp(2*x)/(exp(x) + 1)**2)*(exp(x) + 1)*exp(-x)*log(

(-gamma*y*(1 - exp(x)/(exp(x) + 1))**gamma*(-exp(x)/(exp(x) + 1) + exp(2*x)/(exp(x) + 1)**2)*log(exp(x)/(exp(x) + 1))/(1 - exp(x)/(exp(x) + 1)) - gamma*(exp(x)/(exp(x) + 1))**gamma*(1 - y)*(exp(x)/(exp(x) + 1) - exp(2*x)/(exp(x) + 1)**2)*(exp(x) + 1)*exp(-x)*log(1 - exp(x)/(exp(x) + 1)) - y*(1 - exp(x)/(exp(x) + 1))**gamma*(exp(x)/(exp(x) + 1) - exp(2*x)/(exp(x) + 1)**2)*(exp(x) + 1)*exp(-x) - (exp(x)/(exp(x) + 1))**gamma*(1 - y)*(-exp(x)/(exp(x) + 1) + exp(2*x)/(exp(x) + 1)**2)/(1 - exp(x)/(exp(x) + 1)),
 -gamma**2*y*(1 - exp(x)/(exp(x) + 1))**gamma*(-exp(x)/(exp(x) + 1) + exp(2*x)/(exp(x) + 1)**2)**2*log(exp(x)/(exp(x) + 1))/(1 - exp(x)/(exp(x) + 1))**2 - gamma**2*(exp(x)/(exp(x) + 1))**gamma*(1 - y)*(exp(x)/(exp(x) + 1) - exp(2*x)/(exp(x) + 1)**2)**2*(exp(x) + 1)**2*exp(-2*x)*log(1 - exp(x)/(exp(x) + 1)) - 2*gamma*y*(1 - exp(x)/(exp(x) + 1))**gamma*(-exp(x)/(exp(x) + 1) + exp(2*x)/(exp(x) + 1)**2)*(exp(x)/(exp(x) + 1) - exp(2*x)/(exp(x) + 1)**2)*(exp(x) + 1)*exp(-x)/(1 - exp(x)/(exp

# %%time
# params = config.params
# params.update({
#     'loss_function': FocalLossObjective(),
#     'iterations': iterations,
#     'verbose': iterations // 5,
# })
# clf_logloss_custom_loop = CatBoostClassifier(**params)
# clf_logloss_custom_loop.fit(X=train_pool,
#         eval_set=val_pool,
#         use_best_model=True)
# print(f'Здесь функция потерь logloss реализована через цикл, очень медленно выполняется, \n'
#       f'примерно в 8 раз медленнее, чем если реализовать через numpy \n'
#       f'и в 30 раз медленнее, чем референсная, родная catboost \n'
#       f'также сходится медленнее, чем родная. Сходимость одинаковая с реализацией через numpy ')

In [16]:
# Asymmetric loss ASL
class AsymmetricLossObjective(object):
    def __init__(self, der1, der2, gamma_minus=.2, gamma_plus=.6):
        self.g_minus = gamma_minus
        self.g_plus = gamma_plus
        self.der1 = der1
        self.der2 = der2

    def calc_ders_range(self, x, y, weights):
        assert len(x) == len(y)
        if weights is not None:
            assert len(weights) == len(x)

        e = np.exp(x)
        p = e / (1 + e)

        der1 = self.der1(p=p, y=y, g_minus=self.g_minus, g_plus=self.g_plus)
        der2 = self.der2(p=p, y=y, g_minus=self.g_minus, g_plus=self.g_plus)

        if weights is not None:
            der1 *= weights
            der2 *= weights


        result = list(zip(der1, der2))
        return result



In [17]:
print('Формулы функции потерь обозначены L \n'
      'первая и вторая производные der1 и der2')

ALS_der1, ALS_der2 = AsymmetricLossFormulas()

Формулы функции потерь обозначены L 
первая и вторая производные der1 и der2


Eq(p, exp(x)/(exp(x) + 1))

Eq(L_plus, (1 - exp(x)/(exp(x) + 1))**g_plus*log(exp(x)/(exp(x) + 1)))

Eq(L_minus, (exp(x)/(exp(x) + 1))**g_minus*log(1 - exp(x)/(exp(x) + 1)))

Eq(L, -y*(1 - exp(x)/(exp(x) + 1))**g_plus*log(exp(x)/(exp(x) + 1)) - (exp(x)/(exp(x) + 1))**g_minus*(1 - y)*log(1 - exp(x)/(exp(x) + 1)))

...

Eq(ALS_der2, g_minus**2*p**g_minus*y*log(1 - p) - g_minus**2*p**g_minus*log(1 - p) - 2*g_minus**2*p**(g_minus + 1)*y*log(1 - p) + 2*g_minus**2*p**(g_minus + 1)*log(1 - p) + g_minus**2*p**(g_minus + 2)*y*log(1 - p) - g_minus**2*p**(g_minus + 2)*log(1 - p) + g_minus*p**(g_minus + 1)*y*log(1 - p) - g_minus*p**(g_minus + 1)*log(1 - p) - 3*g_minus*p**(g_minus + 2)*y*log(1 - p) - 2*g_minus*p**(g_minus + 2)*y + 3*g_minus*p**(g_minus + 2)*log(1 - p) + 2*g_minus*p**(g_minus + 2) + 2*g_minus*p**(g_minus + 3)*y*log(1 - p) + 2*g_minus*p**(g_minus + 3)*y - 2*g_minus*p**(g_minus + 3)*log(1 - p) - 2*g_minus*p**(g_minus + 3) - 2*g_minus*p**(g_minus + 2)*y*(1 - p)*log(1 - p)/p - 2*g_minus*p**(g_minus + 2)*y*(1 - p)/p + 2*g_minus*p**(g_minus + 2)*(1 - p)*log(1 - p)/p + 2*g_minus*p**(g_minus + 2)*(1 - p)/p + 2*g_minus*p**(g_minus + 3)*y*(1 - p)*log(1 - p)/p + 2*g_minus*p**(g_minus + 3)*y*(1 - p)/p - 2*g_minus*p**(g_minus + 3)*(1 - p)*log(1 - p)/p - 2*g_minus*p**(g_minus + 3)*(1 - p)/p - g_plus**2*p**2*y*

'*********** sp.factor(der) *************'

Eq(ALS_der2, g_minus**2*p**2*p**g_minus*y*log(1 - p) - g_minus**2*p**2*p**g_minus*log(1 - p) - 2*g_minus**2*p*p**g_minus*y*log(1 - p) + 2*g_minus**2*p*p**g_minus*log(1 - p) + g_minus**2*p**g_minus*y*log(1 - p) - g_minus**2*p**g_minus*log(1 - p) + g_minus*p**2*p**g_minus*y*log(1 - p) + 2*g_minus*p**2*p**g_minus*y - g_minus*p**2*p**g_minus*log(1 - p) - 2*g_minus*p**2*p**g_minus - g_minus*p*p**g_minus*y*log(1 - p) - 2*g_minus*p*p**g_minus*y + g_minus*p*p**g_minus*log(1 - p) + 2*g_minus*p*p**g_minus - g_plus**2*p**2*y*(1 - p)**g_plus*log(p) - g_plus*p**2*y*(1 - p)**g_plus*log(p) - 2*g_plus*p**2*y*(1 - p)**g_plus + g_plus*p*y*(1 - p)**g_plus*log(p) + 2*g_plus*p*y*(1 - p)**g_plus + p**2*p**g_minus*y - p**2*p**g_minus - p**2*y*(1 - p)**g_plus - p*p**g_minus*y + p*p**g_minus + p*y*(1 - p)**g_plus)

'*********** sp.symplify(der) *************'

Eq(ALS_der2, g_minus**2*p**g_minus*y*log(1 - p) - g_minus**2*p**g_minus*log(1 - p) - 2*g_minus**2*p**(g_minus + 1)*y*log(1 - p) + 2*g_minus**2*p**(g_minus + 1)*log(1 - p) + g_minus**2*p**(g_minus + 2)*y*log(1 - p) - g_minus**2*p**(g_minus + 2)*log(1 - p) - g_minus*p**(g_minus + 1)*y*log(1 - p) - 2*g_minus*p**(g_minus + 1)*y + g_minus*p**(g_minus + 1)*log(1 - p) + 2*g_minus*p**(g_minus + 1) + g_minus*p**(g_minus + 2)*y*log(1 - p) + 2*g_minus*p**(g_minus + 2)*y - g_minus*p**(g_minus + 2)*log(1 - p) - 2*g_minus*p**(g_minus + 2) - g_plus**2*p**2*y*(1 - p)**g_plus*log(p) - g_plus*p**2*y*(1 - p)**g_plus*log(p) - 2*g_plus*p**2*y*(1 - p)**g_plus + g_plus*p*y*(1 - p)**g_plus*log(p) + 2*g_plus*p*y*(1 - p)**g_plus - p**2*y*(1 - p)**g_plus + p*y*(1 - p)**g_plus - p**(g_minus + 1)*y + p**(g_minus + 1) + p**(g_minus + 2)*y - p**(g_minus + 2))

In [ ]:
print('1 Упрощенная производная Асимметричной функции потерь ALS \n'
      'упрощение удаляет все экспонеенты, оставляет только логарифмы\n'
      'вторая производная намного длиннее, здесь я её не печатаю ')

ALS_der1_simp = get_simplified_derivative(ALS_der1, verbose=True, der_name='ALS_der1')
ALS_der2_simp = get_simplified_derivative(ALS_der2, verbose=False, der_name='ALS_der2')

In [18]:
%%time
params = config.params
params.update({
    'loss_function': AsymmetricLossObjective(der1=ALS_der1_simp, der2=ALS_der2_simp),
    'iterations': iterations,
    'verbose': iterations // 5,
})
clf_logloss_custom_loop = CatBoostClassifier(**params)
clf_logloss_custom_loop.fit(X=train_pool,
        eval_set=val_pool,
        use_best_model=True)
print(f'Здесь функция потерь ASL реализована через numpy, Python, \n'
      f'формула приводится выше \n'
)



0:	learn: 0.7196265	test: 0.6101332	best: 0.6101332 (0)	total: 28.8s	remaining: 9m 7s
4:	learn: 0.7196265	test: 0.6101332	best: 0.6101332 (0)	total: 2m 15s	remaining: 6m 46s
8:	learn: 0.7261607	test: 0.6160824	best: 0.6160824 (6)	total: 4m 3s	remaining: 4m 57s
12:	learn: 0.7267712	test: 0.6163832	best: 0.6163832 (12)	total: 5m 49s	remaining: 3m 7s
16:	learn: 0.7283695	test: 0.6136529	best: 0.6163832 (12)	total: 7m 35s	remaining: 1m 20s
19:	learn: 0.7283874	test: 0.6136743	best: 0.6163832 (12)	total: 8m 51s	remaining: 0us

bestTest = 0.616383183
bestIteration = 12

Shrink model to first 13 iterations.
Здесь функция потерь ASL реализована через numpy, Python, 
формула приводится выше 

CPU times: user 9min 45s, sys: 45.4 s, total: 10min 31s
Wall time: 8min 52s


In [19]:
print('FINISH')

FINISH
